### Assignment 7.3 : Hotel Recommendations - Python
### Author : Gourav Verma
### DSC630 : Fall-2020

In [1]:
# Import important libraries
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn import svm
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Read train dataset
filename = 'train.csv'
n = sum(1 for line in open(filename)) - 1
s = 100000
skip = sorted(random.sample(range(1, n+1), n-s))
df_train = pd.read_csv(filename, skiprows=skip)

In [3]:
df_train.shape

(100000, 24)

In [4]:
# Write the sample train into another .csv file
df_train.to_csv(r'C:\Users\goura\Desktop\GARV ML\DSC 630\subset_train.csv', index = False)

### subset_train.csv will be used in R for EDA and Data Preparation

#### Refer Hotel_Recommendation_R

### From R merged_prepared_train.csv was created

In [5]:
df = pd.read_csv('Merged_Prepared_Train.csv')

In [6]:
df = df.drop(columns='Unnamed: 0')
df.head()

,date_year,date_month,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,...,d140,d141,d142,d143,d144,d145,d146,d147,d148,d149
0,2013,11,13,1,46,171,15334,860,4539,0,...,-2.248868,-2.248860,-2.248868,-2.248868,-2.248868,-2.248868,-2.248868,-2.248868,-2.248868,-2.248868
1,2014,10,2,3,66,348,48862,307,6304,0,...,-2.324091,-1.033870,-2.067160,-2.329195,-2.330481,-2.325361,-2.170019,-2.329680,-2.330481,-2.320629
2,2014,10,11,3,205,354,25315,1932,6450,0,...,-2.255405,-2.092796,-2.264705,-2.233834,-2.176576,-2.255405,-2.264705,-2.264705,-2.241351,-2.225888
3,2014,3,2,3,66,311,31592,1714,7030,0,...,-2.316944,-2.367790,-2.326313,-2.346321,-2.375115,-2.302345,-2.371526,-2.350070,-2.354576,-2.374588
4,2014,10,2,3,66,153,50542,1745,15632,1,...,-2.274066,-2.239870,-2.273973,-2.288024,-2.284417,-2.259007,-2.299027,-2.281004,-2.290302,-2.299609


In [7]:
relevant_hotel_info = [df.groupby(['srch_destination_id','hotel_country','hotel_market','hotel_cluster'])['is_booking'].agg(['sum','count'])]
agg = pd.concat(relevant_hotel_info).groupby(level=[0,1,2,3]).sum()
#agg.dropna(inplace=True)
agg.head()

sum  count
srch_destination_id hotel_country hotel_market hotel_cluster            
4                   7             246          25               0      1
8                   50            416          16               0      1
                                               23               0      1
                                               43               0      1
                                               48               1      1

In [8]:
agg['sum_and_cnt'] = 0.85*agg['sum'] + 0.15*agg['count']
agg = agg.groupby(level=[0,1,2]).apply(lambda x: x.astype(float)/x.sum())
agg.reset_index(inplace=True)
agg.head()

,srch_destination_id,hotel_country,hotel_market,hotel_cluster,sum,count,sum_and_cnt
0,4,7,246,25,NaN,1.00,1.000000
1,8,50,416,16,0.0,0.25,0.103448
2,8,50,416,23,0.0,0.25,0.103448
3,8,50,416,43,0.0,0.25,0.103448
4,8,50,416,48,1.0,0.25,0.689655


In [9]:
agg_pivot = agg.pivot_table(index=['srch_destination_id','hotel_country','hotel_market'], columns='hotel_cluster', values='sum_and_cnt').reset_index()
agg_pivot.head()

hotel_cluster,srch_destination_id,hotel_country,hotel_market,0,1,2,3,4,5,6,...,90,91,92,93,94,95,96,97,98,99
0,4,7,246,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8,50,416,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,14,27,1434,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,16,50,419,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,19,102,1522,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df = pd.merge(df, agg_pivot, how='left', on=['srch_destination_id','hotel_country','hotel_market'])
df.fillna(0, inplace=True)
df.shape

(63874, 276)

In [11]:
df = df.loc[df['is_booking'] == 1]

# Determing features
X = df.drop(['user_id', 'hotel_cluster', 'is_booking'], axis=1)
y = df.hotel_cluster

In [12]:
# K-Nearest Neighbor Classifier
from sklearn.neighbors import KNeighborsClassifier
clf = make_pipeline(preprocessing.StandardScaler(), KNeighborsClassifier(n_neighbors=5))
np.mean(cross_val_score(clf, X, y, cv=10, scoring='accuracy'))

0.331205362770071

In [13]:
# Random Forest Classifier
clf = make_pipeline(preprocessing.StandardScaler(), RandomForestClassifier(n_estimators=273,max_depth=10,random_state=0))
np.mean(cross_val_score(clf, X, y, cv=10))

0.30886508081669073

In [14]:
# Multi-Class Logistic Regression
from sklearn.linear_model import LogisticRegression
clf = make_pipeline(preprocessing.StandardScaler(), LogisticRegression(multi_class='ovr'))
np.mean(cross_val_score(clf, X, y, cv=10))

0.35755144370473013

In [15]:
# SVM Classifier
from sklearn import svm
clf = make_pipeline(preprocessing.StandardScaler(), svm.SVC(decision_function_shape='ovo'))
np.mean(cross_val_score(clf, X, y, cv=10))

0.38838220796515116